# Comprehensive Stock Technical Analysis

1: Quantitative Analysis with PyNance and TA-Lib

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import talib
import pynance as pn
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("=== QUANTITATIVE FINANCIAL ANALYSIS ===")
print("Libraries imported successfully!")

=== QUANTITATIVE FINANCIAL ANALYSIS ===
Libraries imported successfully!


In [3]:
# Use the symbols you actually have locally
stock_symbols = ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA']
data_folder = "../data/yfinancedata"  

def load_stock_data_from_csv(symbols, folder=data_folder):
    """
    Load stock data for multiple symbols from local CSV files.
    Returns a dict: { 'AAPL': DataFrame, 'AMZN': DataFrame, ... }
    """
    stock_data = {}

    for symbol in symbols:
        file_path = os.path.join(folder, f"{symbol}.csv")
        try:
            print(f"Loading data for {symbol} from {file_path}...")
            df = pd.read_csv(file_path)

            # Expecting columns: Date, Close, High, Low, Open, Volume
            if 'Date' not in df.columns:
                print(f"✗ {symbol}: 'Date' column not found, skipping")
                continue

            # Parse Date and set as index
            df['Date'] = pd.to_datetime(df['Date'], utc=True, errors='coerce')
            df = df.set_index('Date').sort_index()

            if not df.empty:
                stock_data[symbol] = df
                print(f"✓ {symbol}: {len(df)} trading days loaded")
            else:
                print(f"✗ {symbol}: DataFrame is empty after processing")

        except FileNotFoundError:
            print(f"✗ {symbol}: File not found at {file_path}")
        except Exception as e:
            print(f"✗ {symbol}: Error while loading - {e}")

    return stock_data


In [4]:
print("\n" + "="*50)
print("LOADING STOCK DATA FROM CSV")
print("="*50)
stock_data = load_stock_data_from_csv(stock_symbols, folder=data_folder)


LOADING STOCK DATA FROM CSV
Loading data for AAPL from ../data/yfinancedata\AAPL.csv...
✓ AAPL: 3774 trading days loaded
Loading data for AMZN from ../data/yfinancedata\AMZN.csv...
✓ AMZN: 3774 trading days loaded
Loading data for GOOG from ../data/yfinancedata\GOOG.csv...
✓ GOOG: 3774 trading days loaded
Loading data for META from ../data/yfinancedata\META.csv...
✓ META: 2923 trading days loaded
Loading data for MSFT from ../data/yfinancedata\MSFT.csv...
✓ MSFT: 3774 trading days loaded
Loading data for NVDA from ../data/yfinancedata\NVDA.csv...
✓ NVDA: 3774 trading days loaded


In [5]:
# Display data structure for first stock
if stock_data:
    first_symbol = list(stock_data.keys())[0]
    print(f"\nData structure for {first_symbol}:")
    print(stock_data[first_symbol].head())
    print(f"\nColumns: {stock_data[first_symbol].columns.tolist()}")
    print(f"Date range: {stock_data[first_symbol].index.min()} to {stock_data[first_symbol].index.max()}")


Data structure for AAPL:
                              Close      High       Low      Open      Volume
Date                                                                         
2009-01-02 00:00:00+00:00  2.721686  2.730385  2.554037  2.575630   746015200
2009-01-05 00:00:00+00:00  2.836553  2.884539  2.780469  2.794266  1181608400
2009-01-06 00:00:00+00:00  2.789767  2.914229  2.770872  2.877641  1289310400
2009-01-07 00:00:00+00:00  2.729484  2.774170  2.706990  2.753477   753048800
2009-01-08 00:00:00+00:00  2.780169  2.793666  2.700393  2.712090   673500800

Columns: ['Close', 'High', 'Low', 'Open', 'Volume']
Date range: 2009-01-02 00:00:00+00:00 to 2023-12-29 00:00:00+00:00


2: Calculate Technical Indicators with TA-Lib

In [6]:
def calculate_technical_indicators(df):
    """
    Calculate comprehensive technical indicators for a stock DataFrame
    """
    indicators_df = df.copy()
    
    # Price-based indicators
    indicators_df['SMA_20'] = talib.SMA(indicators_df['Close'], timeperiod=20)
    indicators_df['SMA_50'] = talib.SMA(indicators_df['Close'], timeperiod=50)
    indicators_df['SMA_200'] = talib.SMA(indicators_df['Close'], timeperiod=200)
    
    indicators_df['EMA_12'] = talib.EMA(indicators_df['Close'], timeperiod=12)
    indicators_df['EMA_26'] = talib.EMA(indicators_df['Close'], timeperiod=26)
    
    # Momentum Indicators
    indicators_df['RSI_14'] = talib.RSI(indicators_df['Close'], timeperiod=14)
    indicators_df['RSI_21'] = talib.RSI(indicators_df['Close'], timeperiod=21)
    
    # MACD
    indicators_df['MACD'], indicators_df['MACD_Signal'], indicators_df['MACD_Hist'] = talib.MACD(
        indicators_df['Close'], fastperiod=12, slowperiod=26, signalperiod=9
    )
    
    # Bollinger Bands
    indicators_df['BB_Upper'], indicators_df['BB_Middle'], indicators_df['BB_Lower'] = talib.BBANDS(
        indicators_df['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0
    )
    
    # Volatility Indicators
    indicators_df['ATR_14'] = talib.ATR(
        indicators_df['High'], indicators_df['Low'], indicators_df['Close'], timeperiod=14
    )
    
    # Volume Indicators
    indicators_df['Volume_SMA_20'] = talib.SMA(indicators_df['Volume'], timeperiod=20)
    
    # Stochastic
    indicators_df['SlowK'], indicators_df['SlowD'] = talib.STOCH(
        indicators_df['High'], indicators_df['Low'], indicators_df['Close'],
        fastk_period=14, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0
    )
    
    # Additional indicators
    indicators_df['ADX_14'] = talib.ADX(
        indicators_df['High'], indicators_df['Low'], indicators_df['Close'], timeperiod=14
    )
    
    indicators_df['OBV'] = talib.OBV(indicators_df['Close'], indicators_df['Volume'])
    
    return indicators_df

print("\n" + "="*50)
print("CALCULATING TECHNICAL INDICATORS")
print("="*50)

# Calculate indicators for all stocks
stock_indicators = {}
for symbol, data in stock_data.items():
    print(f"Calculating indicators for {symbol}...")
    stock_indicators[symbol] = calculate_technical_indicators(data)

# Display indicators for first stock
if stock_indicators:
    first_symbol = list(stock_indicators.keys())[0]
    indicator_cols = [col for col in stock_indicators[first_symbol].columns if col not in ['Open', 'High', 'Low', 'Close', 'Volume']]
    
    print(f"\nTechnical indicators calculated for {first_symbol}:")
    print(f"Total indicators: {len(indicator_cols)}")
    print("Indicator columns:")
    for i, col in enumerate(indicator_cols, 1):
        print(f"{i:2d}. {col}")


CALCULATING TECHNICAL INDICATORS
Calculating indicators for AAPL...
Calculating indicators for AMZN...
Calculating indicators for GOOG...
Calculating indicators for META...
Calculating indicators for MSFT...
Calculating indicators for NVDA...

Technical indicators calculated for AAPL:
Total indicators: 19
Indicator columns:
 1. SMA_20
 2. SMA_50
 3. SMA_200
 4. EMA_12
 5. EMA_26
 6. RSI_14
 7. RSI_21
 8. MACD
 9. MACD_Signal
10. MACD_Hist
11. BB_Upper
12. BB_Middle
13. BB_Lower
14. ATR_14
15. Volume_SMA_20
16. SlowK
17. SlowD
18. ADX_14
19. OBV
